In [1]:
# load dataset data/test.csv and data/train.csv

import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(train.shape, test.shape)

train.head()

(120000, 2) (7600, 2)


,text,label
0,wall st bear claw back black reuters reuters s...,2
1,carlyle look toward commercial aerospace reute...,2
2,oil economy cloud stock outlook reuters reuter...,2
3,iraq halt oil export main southern pipeline re...,2
4,oil price soar alltime record posing new menac...,2


In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
model.cuda()

/data/tianyang/anaconda3/envs/repobench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
# Custom dataset class
from torch.utils.data import Dataset

class AGNewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import wandb


# Initialize Weights & Biases
wandb.init(project="text_classification", name="bert-base")

# Create dataset (assuming train and test DataFrames are already defined)
train_texts = train['text'].tolist()
train_labels = train['label'].tolist()
test_texts = test['text'].tolist()
test_labels = test['label'].tolist()

train_dataset = AGNewsDataset(train_texts, train_labels)
test_dataset = AGNewsDataset(test_texts, test_labels)

# Create dataloader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Define model, optimizer, scheduler, and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def evaluate_model(model, test_loader):
    model.eval()
    total_loss, total_accuracy = 0, 0

    with torch.no_grad():
        for texts, labels in test_loader:
            inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
            labels = torch.tensor(labels).to(device)

            outputs = model(**inputs, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            total_accuracy += (outputs.logits.argmax(dim=-1) == labels).sum().item()

    return total_loss / len(test_loader), total_accuracy / len(test_loader.dataset)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for step, (texts, labels) in progress_bar:
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
        labels = torch.tensor(labels).to(device)

        optimizer.zero_grad()

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        wandb.log({"train_loss": loss.item()})

        # Update progress bar
        progress_bar.set_postfix({"Train Loss": f"{running_loss/(step+1):.4f}"})

        if step % 100 == 0:
            # Evaluate on test set every 100 steps
            test_loss, test_accuracy = evaluate_model(model, test_loader)
            wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})
            progress_bar.set_postfix({"Train Loss": f"{running_loss/(step+1):.4f}", "Test Loss": f"{test_loss:.4f}", "Test Acc": f"{test_accuracy:.4f}"})

    progress_bar.close()

# Final evaluation on test set
final_test_loss, final_test_accuracy = evaluate_model(model, test_loader)
wandb.log({"final_test_loss": final_test_loss, "final_test_accuracy": final_test_accuracy})

# Finish the wandb run
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leolty. Use `wandb login --relogin` to force relogin


/data/tianyang/anaconda3/envs/repobench/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/3:   0%|          | 0/938 [00:00<?, ?it/s]/tmp/ipykernel_3425928/2516463646.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)
Epoch 1/3:   0%|          | 0/938 [00:00<?, ?it/s, Train Loss=1.4195]/tmp/ipykernel_3425928/2516463646.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(la

final_test_accuracy,▁
final_test_loss,▁
test_accuracy,▁█████████████████████████████
test_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▄▄▄▃▄▃▂▄▄▃▃▃▂▃▂▂▃▁▂▂▃▃▂▂▁▃▃▁▁▂▁▁▂▂▁▂▂▂▂
final_test_accuracy,0.93013
final_test_loss,0.20936
test_accuracy,0.92987
test_loss,0.2094
train_loss,0.11451


In [ ]:
# Save model
model.save_pretrained('checkpoints/bert-base')